# [개념 필사] - 파머완

## CHAPTER 09 <추천 시스템>

## **9.7 행렬 분해를 이용한 잠재요인 협업 필터링 실습**

✅ 사용자-아이템 행렬, NULL 많음 ➡ SGD, ALS 기반 행렬분해 이용

✅ 4절에서의 함수 코드 사용
* get_rmse()
* 행렬분해로직을 정리하는 matrix_funtion(평점행렬, 잠재요인차원수, SGD 반복횟수, 학습률, L2규제계수)

> 🟠 영화 평점 행렬 데이터 🟠

(1) DATA LOAD

(2) SGD 활용 행렬 분해

* matrix_funtion() 함수 사용

(3) 영화 추천 수행
* get_unseen_movies() 사용
* recomm_movie_by_userid() 사용


## **9.8 파이썬 추천 시스템 패키지 - Surprise**

◼ Surprise 패키지: 파이썬 기반의 추천 시스템 구축을 위한 전용 패키지. 사이킷런과 유사한 API, 프레임워크 제공

* 다양한 추천 알고리즘 쉽게 구축 가능
* 사이킷런과 유사한 API명

> 🟠 Surprise 를 이용한 시스템 구축 🟠

◼ SVD 행렬 분해를 통한 잠재 요인 협업필터링 수행

(1) DATA LOAD

* 💻 load_builtin() - 무비렌즈 사이트에서 제공하는 과거버전 데이터세트를 로딩. (탭으로 분리)

* 학습, 테스트 데이터세트 분리

(2) SVD 잠재 요인 협업 필터링 수행

✅ Surprise는 자체적으로 로우 레벨을 칼럼레벨 데이터로 변경 ➡ 로우레벨의 원본 데이터를 그대로 적용해야함.


* 추천 예측 메서드
```
test() - 평점데이터 전체에 대해서 추천 예측

predict() - 개별 사용자,영화에 대한 추천 평점 반환
```

(3) 예측 평가

```
accuracy.rmse
```

> **주요 모듈 소개**

* 데이터의 칼럼을 1.사용자 아이디, 2.아이템 아이디, 3.평점으로 가정하고 이외는 로딩 수행X(순서 중요)

* API: Dataset.load_builtin(name='m1-100k'): 무비렌즈 아카이브 데이터, Dataset.load_from_file(file_path, reader):OS 파일에서 로딩, Dataset.load_from_df(df, reader): 판다스의 DataFrame에서 데이터 로딩

* OS 파일 데이터 -> Surprise 데이터

  -- 칼럼명 헤더 삭제필요: header=False로 새 파일 저장 후 다시 로드
  
  -- Reader() 클래스 생성: 파일의 파싱정보 저장. 각 필드의 칼럼명(line_format), 칼럼 분리문자(sep), 최소~최대 평점(rating_scale) (line_format='user item ... ': 칼럼명 명시)

  -- Dataset.load_from_file('df경로',reader=reader)

  -- SVD(n_factors=50): 잠재요인 크기 K값=50으로 설정

  -- 학습데이터로 학습, 테스트 데이터로 예측평점 구한 후 RMSE 평가

* 판다스 DataFrame -> Surprise 데이터

  -- Dataset.load_from_df(data[['userId','movieId','rating']], reader=reader)

  -- 칼럼 순서 잘 지키기. df[['userId','movieId','rating']]


> **추천 알고리즘 클래스**

* SVD: 행렬분해 잠재요인 협업필터링을 위한 SVD 알고리즘

* 파라미터: n_factors(잠재요인 K개수), n_epochs(SGD 수행시 반복 횟수), biased(베이스라인 사용자 편향 적용 여부)

* KNNBasic: 최근접 이웃협업필터링을 위한 KNN 알고리즘

* BaselineOnly: 사용자 Bias와 아이템 Bias를 감안한 SGD 베이스라인 알고리즘


💡 (벤치마크 결과) SVD++가 가장 성적이 좋지만 시간이 너무 오래걸림. SVD와 k-NN+Baseline이 다음으로 좋음.


◼ **베이스라인 평점**: 개인의 성향을 반영해 아이템 평가에 편향성 요소를 반영하여 평점을 부과하는 것.

* 전체 평균 평점 + 사용자 편향 점수 + 아이템 편향 점수

> 교차 검증, 하이퍼 파라미터 튜닝

* 💻 cross_validate() - 인자는 알고리즘 객체, 데이터, 성능평가방법, 폴드 데이터 세트 개수

* 💻 GridSearchCV

  -- SVD, 주로 n_epochs, n_factors 튜닝


> 🟠 Surprise - 영화 평점 행렬 데이터 🟠

◼ Surprise 학습된 추천 알고리즘을 기반으로 특정 사용자가 아직 평점을 매기지 않은 영화 중에서 취향에 가장 적절한 영화 추천하기.

✅ 데이터세트 분리 X ➡ fit() ➡ 오류 발생

💡 "💻 DatasetAutoFolds 클래스"
```
build_full_trainset() - 전체데이터를 학습데이터세트로 만들기 가능
```

i) 사용자 9가 아직 평점을 매기지않은 movie42에 대해 예측 평점 계산 (predict() 메서드 사용)

ii) 사용자가 평점을 매기지 않은 전체 영화 추출, 예측 평점 순으로 영화 추천: get_unseen_surprise(), recomm_movie_by_surprise() 함수 생성


# [코드 필사]

In [3]:
# get_rmse()함수
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
  error = 0
  # 두 개의 분해된 행렬 P, Q.T 내적으로 예측 R 행렬 생성
  full_pred_matrix = np.dot(P, Q.T)

  # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출해 실제 R 행렬과 예측 행렬의 RMSE 추출
  x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
  y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
  R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
  full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
  mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
  rmse = np.sqrt(mse)

  return rmse

In [4]:
def matrix_function(R, k, steps=200, learning_rate=0.01, r_lambda=0.01):
  num_users, num_items = R.shape
  # P와 Q 매트릭스의 크기를 지정하고 정규 분포를 가진 랜덤한 값으로 입력
  np.random.seed(1)
  P = np.random.normal(scale=1./k, size=(num_users, k))
  Q = np.random.normal(scale=1./k, size=(num_items, k))

  # R>0인 행 위치, 열 위치 값을 non_zeros 리스트 객체에 저장
  non_zeros = [(i, j, R[i, j]) for i in range(num_users) for j in range(num_items) if R[i, j] > 0]

  # SGD 기법으로 P,Q 를 계쏙 업데이트
  for step in range(steps):
    for i, j, r in non_zeros:
      # 실제 값과 예측 값의 차이인 오류 값 구함
      eij = r - np.dot(P[i,:],Q[j,:].T)
      # 정규화를 반영한 SGD 업데이트 공식 적용
      P[i,:] = P[i,:] + learning_rate*(eij * Q[j,:] - r_lambda*P[i,:])
      Q[j,:] = Q[j,:] + learning_rate*(eij * P[i,:] - r_lambda*Q[j,:])

    rmse = get_rmse(R, P, Q, non_zeros)
    if (step % 10) == 0:
      print('### iteration step: ', step, ' rmse: ', rmse)

  return P, Q

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
import pandas as pd
import numpy as np

movies = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/movies.csv')
ratings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/ratings.csv',header=None, names=['userId', 'movieId', 'rating', 'timestamp'])
ratings = ratings[['userId', 'movieId', 'rating']]
ratings_matrix = ratings.pivot_table('rating', index='userId', columns='movieId')

# title 칼럼을 얻기 위해 movies와 조인 수행
rating_movies = pd.merge(ratings, movies, on='movieId')

# columns='title'로 title 칼럼으로 피벗 수행
ratings_matrix = rating_movies.pivot_table('rating', index='userId', columns='title')

In [31]:
# 행렬분해
P, Q = matrix_function(ratings_matrix.values, k=50, steps=200, learning_rate=0.01, r_lambda=0.01)
pred_matrix = np.dot(P, Q.T)

### iteration step:  0  rmse:  2.9023619751336867
### iteration step:  10  rmse:  0.7335768591017927
### iteration step:  20  rmse:  0.5115539026853442
### iteration step:  30  rmse:  0.37261628282537446
### iteration step:  40  rmse:  0.2960818299181014
### iteration step:  50  rmse:  0.2520353192341642
### iteration step:  60  rmse:  0.22487503275269854
### iteration step:  70  rmse:  0.2068545530233154
### iteration step:  80  rmse:  0.19413418783028685
### iteration step:  90  rmse:  0.18470082002720406
### iteration step:  100  rmse:  0.17742927527209104
### iteration step:  110  rmse:  0.1716522696470749
### iteration step:  120  rmse:  0.16695181946871726
### iteration step:  130  rmse:  0.16305292191997542
### iteration step:  140  rmse:  0.15976691929679646
### iteration step:  150  rmse:  0.1569598699945732
### iteration step:  160  rmse:  0.15453398186715425
### iteration step:  170  rmse:  0.15241618551077643
### iteration step:  180  rmse:  0.1505508073962831
### iteration

In [32]:
# 데이터프레임으로 변경
ratings_pred_matrix = pd.DataFrame(data=pred_matrix, index=ratings_matrix.index,
                                   columns=ratings_matrix.columns)
ratings_pred_matrix.head(3)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,3.055084,4.092018,3.564130,4.502167,3.981215,1.271694,3.603274,2.333266,5.091749,3.972454,...,1.402608,4.208382,3.705957,2.720514,2.787331,3.475076,3.253458,2.161087,4.010495,0.859474
2,3.170119,3.657992,3.308707,4.166521,4.311890,1.275469,4.237972,1.900366,3.392859,3.647421,...,0.973811,3.528264,3.361532,2.672535,2.404456,4.232789,2.911602,1.634576,4.135735,0.725684
3,2.307073,1.658853,1.443538,2.208859,2.229486,0.780760,1.997043,0.924908,2.970700,2.551446,...,0.520354,1.709494,2.281596,1.782833,1.635173,1.323276,2.887580,1.042618,2.293890,0.396941


In [33]:
# 사용자 9번 영화추천
def get_unseen_movies(ratings_matrix, userId):
    # userId로 입력받은 사용자의 모든 영화 정보를 추출해 Series로 반환함.
    # 반환된 user_rating은 영화명(title)을 인덱스를 가지는 Series 객체임.
    user_rating = ratings_matrix.loc[userId, :]

    #user_rating이 0보다 크면 기존에 관람한 영화임. 대상 인덱스를 추출해 list 객체로 만듦.
    already_seen = user_rating[ user_rating > 0].index.tolist()

    # 모든 영화명을 list 객체로 만듦.
    movies_list = ratings_matrix.columns.tolist()

    # list comporehension으로 already_seen에 해당하는 영화는 movies_list에서 제외함.
    unseen_list = [ movie for movie in movies_list if movie not in already_seen]

    return unseen_list

In [34]:
def recomm_movie_by_userid(pred_df, userId, unseen_list, top_n = 10):
    # 예측 평점 DataFrame에서 사용자id 인덱스와 unseen_list로 들어온 영화명 칼럼을 추출해
    # 가장 예측 평점이 높은 순으로 정렬함.
    recomm_movies = pred_df.loc[userId, unseen_list].sort_values(ascending=False)[:top_n]
    return recomm_movies

In [35]:
# 사용자가 관람하지 않은 영화명 추출
unseen_list = get_unseen_movies(ratings_matrix, 9)

# 아이템 기반의 최근접 이웃 협업 필터링으로 영화 추천
recomm_movies = recomm_movie_by_userid(ratings_pred_matrix, 9, unseen_list, top_n=10)

# 평점 데이터를 DataFrame으로 생성.
recomm_movies = pd.DataFrame(data=recomm_movies.values, index=recomm_movies.index)
columns = ['pred_score']
recomm_movies

,0
title,
Rear Window (1954),5.704612
"South Park: Bigger, Longer and Uncut (1999)",5.451100
Rounders (1998),5.298393
Blade Runner (1982),5.244951
Roger & Me (1989),5.191962
Gattaca (1997),5.183179
Ben-Hur (1959),5.130463
Rosencrantz and Guildenstern Are Dead (1990),5.087375
"Big Lebowski, The (1998)",5.038690


## Surprise

In [2]:
!pip install scikit-surprise

  Using cached scikit_surprise-1.1.4-cp312-cp312-linux_x86_64.whl


In [2]:
import surprise
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [7]:
data = Dataset.load_builtin('ml-100k')
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [8]:
algo = SVD(random_state=0)
algo.fit(trainset)

In [9]:
predictions = algo.test(testset)
print('prediction type: ', type(predictions), 'size: ', len(predictions))
print('predicton 결과의 최초 5개 추출')
predictions[:5]

prediction type:  <class 'list'> size:  25000
predicton 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.5114147666251547, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.573872419581491, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.033583485472447, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.8463639495936905, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.1807542478219157, details={'was_impossible': False})]

* Prediction 객체: 튜플 형태로 데이터 가지고 있음

   -- datails 속성: 추천 예측이 불가능한 경우, 로그용으로 데이터를 남기는 데 사용 (was_impossible=True)

In [10]:
# 객체에서 속성 추출
[(pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('120', '282', 3.5114147666251547),
 ('882', '291', 3.573872419581491),
 ('535', '507', 4.033583485472447)]

In [11]:
# 기존 평점 정보는 선택사항 / 사용자 아이디,아이템아이디는 문자열로 입력해야 함.
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.49   {'was_impossible': False}


In [12]:
accuracy.rmse(predictions)

RMSE: 0.9467


0.9466860806937948



---



In [14]:
import pandas as pd

ratings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/ratings.csv')
# # ratings_noh.csv 파일로 언로드 시 인덱스와 헤더를 모두 제거한 새로운 파일 생성
ratings.to_csv('/content/drive/MyDrive/Colab Notebooks/data/ratings_noh.csv', index=False, header=False)

In [15]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data = Dataset.load_from_file('/content/drive/MyDrive/Colab Notebooks/data/ratings_noh.csv', reader=reader)

In [16]:
trainset, testset  = train_test_split(data, test_size=25, random_state=0)

# 수행 시마다 동일한 결과를 도출하기 위해 random_state 설정
algo = SVD(n_factors=50, random_state=0)

# 학습 데이터 세트로 학습하고 나서 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 1.0051


1.005068647071627

* 판다스 데이터프레임 ➡ surprise 데이터세트

In [18]:
import pandas as pd
from surprise import Reader, Dataset

# Load the CSV without a header and specify column names
ratings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/ratings.csv', header=None, names=['userId', 'movieId', 'rating', 'timestamp'])

# Drop the timestamp column as it's not needed for the Surprise dataset
ratings = ratings[['userId', 'movieId', 'rating']]

reader = Reader(rating_scale=(0.5, 5.0))

# ratings DataFrame에서 칼럼은 사용자 아이디, 아이템 아이디, 평점 순서를 지켜야함.
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516



---



* 교차검증, 성능 평가

In [19]:
from surprise.model_selection import cross_validate

# Pandas DataFrame에서 Surprise 데이터 세트로 데이터 로딩
ratings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/ratings.csv', header=None, names=['userId', 'movieId', 'rating', 'timestamp'])
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8634  0.8808  0.8777  0.8776  0.8659  0.8731  0.0070  
MAE (testset)     0.6627  0.6752  0.6734  0.6757  0.6668  0.6708  0.0051  
Fit time          1.82    2.18    1.49    1.57    1.46    1.70    0.27    
Test time         0.19    0.46    0.12    0.29    0.13    0.24    0.13    


{'test_rmse': array([0.86338278, 0.88081802, 0.87768527, 0.87756245, 0.86590924]),
 'test_mae': array([0.66268041, 0.6752375 , 0.67344632, 0.67566547, 0.66684013]),
 'fit_time': (1.823906421661377,
  2.1830475330352783,
  1.4852230548858643,
  1.5665147304534912,
  1.4628210067749023),
 'test_time': (0.19000720977783203,
  0.4649224281311035,
  0.12118268013000488,
  0.2902350425720215,
  0.12966275215148926)}

* 하이퍼 파라미터 튜닝

In [20]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터를 딕셔너리 형태로 지정
param_grid = {'n_epochs': [20,40,60], 'n_factors': [50,100,200]}

# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse로 수행하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse','mae'], cv=3)
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8770977719067033
{'n_epochs': 20, 'n_factors': 50}




---



In [21]:
# 다음 코드는 train_test_split()으로 분리되지 않는 데이터 세트에 fit()을 호출해 오류가 발생함
data = Dataset.load_from_df(ratings[['userId','movieId','rating']], reader)
algo = SVD(n_factors=50, random_state=0)
algo.fit(data)

AttributeError: 'DatasetAutoFolds' object has no attribute 'n_users'

In [22]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성
data_folds = DatasetAutoFolds(ratings_file='/content/drive/MyDrive/Colab Notebooks/data/ratings_noh.csv', reader=reader)

# 전체 데이터를 학습 데이터로 생성함
trainset = data_folds.build_full_trainset()

In [23]:
# 학습 수행
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

In [25]:
# 영화에 대한 상세 속성 정보 DataFrame 로딩
movies = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/movies.csv')

# userId=9의 movieID 데이터를 추출해 movieI=42 데이터가 있는지 확인.
movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==42].count()==0:
  print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [26]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.10   {'was_impossible': False}


In [27]:
def get_unseen_surprise(ratings, movies, userId):
  #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
  seen_movies = ratings[ratings['userId']==userId]['movieId'].tolist()

  # 모든 영화의 moiveId를 리스트로 생성
  total_movies = movies['movieId'].tolist()

  # 모든 영화의 movieId 중 이미 평점을 매긴 영화의 movieId를 제외한 후 리스트로 생성
  unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
  print('평점 매긴 영화 수: ', len(seen_movies), '추천 대상 영화 수: ', len(unseen_movies),
        '전체 영화 수: ', len(total_movies))

  return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화 수:  46 추천 대상 영화 수:  9696 전체 영화 수:  9742


In [28]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):

  # 알고리즘 객체의 predict()매서드를 평점이 없는 영화에 반복 수행 후 결과를 list 객체로 저장
  predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]

  # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음.
  # [Prediction(uid='9', iid='1', est=3.69), Predcition(uid='9',iid='2',est=2.98),,,,]

  # 이를 est 값으로 정렬하기 위해 아래의 sortkey_est 함수를 정의
  # sortkey_est 함수는 list 객체의 sort()함수의 키 값으로 사용되어 정렬 수행
  def sortkey_est(pred):
    return pred.est

  # sortkey_est() 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출
  predictions.sort(key=sortkey_est, reverse=True)
  top_predictions = predictions[:top_n]

  # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
  top_movie_ids = [int(pred.iid) for pred in top_predictions]
  top_movie_rating = [pred.est for pred in top_predictions]
  top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']

  top_movie_preds = [(id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]

  return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)

print('##### Top-10 추천 영화 리스트 #####')

for top_movie in top_movie_preds:
  print(top_movie[1], ':', top_movie[2])

평점 매긴 영화 수:  46 추천 대상 영화 수:  9696 전체 영화 수:  9742
##### Top-10 추천 영화 리스트 #####
Usual Suspects, The (1995) : 4.2267942523743605
Star Wars: Episode IV - A New Hope (1977) : 4.211016328181448
Shawshank Redemption, The (1994) : 4.197037380337041
Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964) : 4.142771982890766
Godfather, The (1972) : 4.136604760890598
Reservoir Dogs (1992) : 4.124673084995863
Streetcar Named Desire, A (1951) : 4.120523998974084
Goodfellas (1990) : 4.069435046627849
Glory (1989) : 4.067497992056492
All the President's Men (1976) : 4.062778702133245
